In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
df_credit = pd.read_csv(r"C:\Users\rtc32\OneDrive\바탕 화면\데이톤\Datasets\credit_card_balance.csv")
df_install = pd.read_csv(r"C:\Users\rtc32\OneDrive\바탕 화면\데이톤\Datasets\installments_payments.csv")

In [3]:
df_credit.shape

(3840312, 23)

In [7]:
def count_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outlier_mask = (df[col] < lower) | (df[col] > upper)
    outlier_count = outlier_mask.sum()
    mean_all = df[col].mean()
    mean_outlier = df.loc[outlier_mask, col].mean()
    mean_non_outlier = df.loc[~outlier_mask, col].mean()
    print(f"{col} 컬럼의 IQR 기준 극단치 행 개수: {outlier_count}")
    print(f"전체 평균: {mean_all:.2f}")
    print(f"극단치 평균: {mean_outlier:.2f}")
    print(f"극단치 제외 평균: {mean_non_outlier:.2f}")
    print(f"전체 평균과 극단치 제외 평균 차이: {abs(mean_all - mean_non_outlier):.2f}")
    print("-" * 40)

# 모든 수치형 컬럼에 대해 반복
for col in df_credit.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns:
    count_outliers_iqr(df_credit, col)

SK_ID_PREV 컬럼의 IQR 기준 극단치 행 개수: 0
전체 평균: 1904503.59
극단치 평균: nan
극단치 제외 평균: 1904503.59
전체 평균과 극단치 제외 평균 차이: 0.00
----------------------------------------
SK_ID_CURR 컬럼의 IQR 기준 극단치 행 개수: 0
전체 평균: 278324.21
극단치 평균: nan
극단치 제외 평균: 278324.21
전체 평균과 극단치 제외 평균 차이: 0.00
----------------------------------------
MONTHS_BALANCE 컬럼의 IQR 기준 극단치 행 개수: 0
전체 평균: -34.52
극단치 평균: nan
극단치 제외 평균: -34.52
전체 평균과 극단치 제외 평균 차이: 0.00
----------------------------------------
AMT_BALANCE 컬럼의 IQR 기준 극단치 행 개수: 248146
전체 평균: 58300.16
극단치 평균: 370578.32
극단치 제외 평균: 36728.05
전체 평균과 극단치 제외 평균 차이: 21572.10
----------------------------------------
AMT_CREDIT_LIMIT_ACTUAL 컬럼의 IQR 기준 극단치 행 개수: 404927
전체 평균: 153807.96
극단치 평균: 546655.42
극단치 제외 평균: 107503.24
전체 평균과 극단치 제외 평균 차이: 46304.72
----------------------------------------
AMT_DRAWINGS_ATM_CURRENT 컬럼의 IQR 기준 극단치 행 개수: 424778
전체 평균: 5961.32
극단치 평균: 43371.95
극단치 제외 평균: 0.00
전체 평균과 극단치 제외 평균 차이: 5961.32
----------------------------------------
AMT_DRAWINGS_CURRENT 컬럼의 IQR 기준 

In [8]:
exclude_cols = ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']
numeric_cols = [col for col in df_credit.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns if col not in exclude_cols]

df_credit_clean = df_credit.copy()
for col in numeric_cols:
    Q1 = df_credit_clean[col].quantile(0.25)
    Q3 = df_credit_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outlier_mask = (df_credit_clean[col] < lower) | (df_credit_clean[col] > upper)
    df_credit_clean = df_credit_clean[~outlier_mask]

print(df_credit_clean.shape)

(1855163, 23)


In [9]:
exclude_cols = ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']
numeric_cols = [col for col in df_credit.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns if col not in exclude_cols]

df_credit_top1_clean = df_credit.copy()
for col in numeric_cols:
    upper = df_credit_top1_clean[col].quantile(0.99)
    df_credit_top1_clean = df_credit_top1_clean[df_credit_top1_clean[col] <= upper]

print(df_credit_top1_clean.shape)

(2590425, 23)


In [4]:
exclude_cols = exclude_cols = ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']
numeric_cols = [col for col in df_credit.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns if col not in exclude_cols]

print("=== IQR(사분위) 방식 ===")
for col in numeric_cols:
    Q1 = df_credit[col].quantile(0.25)
    Q3 = df_credit[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outlier_mask = (df_credit[col] < lower) | (df_credit[col] > upper)
    mean_all = df_credit[col].mean()
    mean_non_outlier = df_credit.loc[~outlier_mask, col].mean()
    print(f"{col} 전체 평균: {mean_all:.2f}, 극단치 제외 평균: {mean_non_outlier:.2f}, 차이: {abs(mean_all - mean_non_outlier):.2f}")

print("\n=== 상위 1% 제거 방식 ===")
for col in numeric_cols:
    upper = df_credit[col].quantile(0.99)
    mean_all = df_credit[col].mean()
    mean_top1_removed = df_credit[df_credit[col] <= upper][col].mean()
    print(f"{col} 전체 평균: {mean_all:.2f}, 상위 1% 제외 평균: {mean_top1_removed:.2f}, 차이: {abs(mean_all - mean_top1_removed):.2f}")

=== IQR(사분위) 방식 ===
AMT_BALANCE 전체 평균: 58300.16, 극단치 제외 평균: 36728.05, 차이: 21572.10
AMT_CREDIT_LIMIT_ACTUAL 전체 평균: 153807.96, 극단치 제외 평균: 107503.24, 차이: 46304.72
AMT_DRAWINGS_ATM_CURRENT 전체 평균: 5961.32, 극단치 제외 평균: 0.00, 차이: 5961.32
AMT_DRAWINGS_CURRENT 전체 평균: 7433.39, 극단치 제외 평균: 0.00, 차이: 7433.39
AMT_DRAWINGS_OTHER_CURRENT 전체 평균: 288.17, 극단치 제외 평균: 0.00, 차이: 288.17
AMT_DRAWINGS_POS_CURRENT 전체 평균: 2968.80, 극단치 제외 평균: 0.00, 차이: 2968.80
AMT_INST_MIN_REGULARITY 전체 평균: 3540.20, 극단치 제외 평균: 2800.62, 차이: 739.58
AMT_PAYMENT_CURRENT 전체 평균: 10280.54, 극단치 제외 평균: 4124.71, 차이: 6155.83
AMT_PAYMENT_TOTAL_CURRENT 전체 평균: 7588.86, 극단치 제외 평균: 2551.22, 차이: 5037.64
AMT_RECEIVABLE_PRINCIPAL 전체 평균: 55965.88, 극단치 제외 평균: 35012.66, 차이: 20953.22
AMT_RECIVABLE 전체 평균: 58088.81, 극단치 제외 평균: 36631.54, 차이: 21457.27
AMT_TOTAL_RECEIVABLE 전체 평균: 58098.29, 극단치 제외 평균: 36642.26, 차이: 21456.03
CNT_DRAWINGS_ATM_CURRENT 전체 평균: 0.31, 극단치 제외 평균: 0.00, 차이: 0.31
CNT_DRAWINGS_CURRENT 전체 평균: 0.70, 극단치 제외 평균: 0.00, 차이: 0.70
CNT_DRAWINGS_

In [5]:
exclude_cols = ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']
numeric_cols = [col for col in df_credit.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns if col not in exclude_cols]

outlier_ids = set()
for col in numeric_cols:
    upper = df_credit[col].quantile(0.99)
    ids = df_credit.loc[df_credit[col] > upper, 'SK_ID_CURR']
    outlier_ids.update(ids.tolist())

outlier_ids = list(outlier_ids)
print(f"상위 1%에 해당하는 SK_ID_CURR 개수: {len(outlier_ids)}")
print(outlier_ids[:10])  # 일부만 출력

상위 1%에 해당하는 SK_ID_CURR 개수: 50239
[393220, 131079, 131080, 262152, 262156, 131085, 393233, 393238, 262166, 131095]


In [6]:
exclude_cols = ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']
numeric_cols = [col for col in df_credit.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns if col not in exclude_cols]

outlier_dict = {}

for col in numeric_cols:
    upper = df_credit[col].quantile(0.99)
    ids = df_credit.loc[df_credit[col] > upper, 'SK_ID_CURR'].unique()
    outlier_dict[col] = ids

# 데이터프레임으로 변환 (컬럼별로 아이디를 리스트로 저장)
outlier_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in outlier_dict.items()]))

# csv로 저장
outlier_df.to_csv("credit_outlier_ids_by_column.csv", index=False)

In [10]:
df_install.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')

In [11]:
df_install.dtypes

SK_ID_PREV                  int64
SK_ID_CURR                  int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER       int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtype: object

In [12]:
exclude_cols = ['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION']
numeric_cols = [col for col in df_install.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns if col not in exclude_cols]

print("=== IQR(사분위) 방식 ===")
for col in numeric_cols:
    Q1 = df_install[col].quantile(0.25)
    Q3 = df_install[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outlier_mask = (df_install[col] < lower) | (df_install[col] > upper)
    mean_all = df_install[col].mean()
    mean_non_outlier = df_install.loc[~outlier_mask, col].mean()
    print(f"{col} 전체 평균: {mean_all:.2f}, 극단치 제외 평균: {mean_non_outlier:.2f}, 차이: {abs(mean_all - mean_non_outlier):.2f}")

print("\n=== 상위 1% 제거 방식 ===")
for col in numeric_cols:
    upper = df_install[col].quantile(0.99)
    mean_all = df_install[col].mean()
    mean_top1_removed = df_install[df_install[col] <= upper][col].mean()
    print(f"{col} 전체 평균: {mean_all:.2f}, 상위 1% 제외 평균: {mean_top1_removed:.2f}, 차이: {abs(mean_all - mean_top1_removed):.2f}")

=== IQR(사분위) 방식 ===
NUM_INSTALMENT_NUMBER 전체 평균: 18.87, 극단치 제외 평균: 9.54, 차이: 9.33
DAYS_INSTALMENT 전체 평균: -1042.27, 극단치 제외 평균: -1042.27, 차이: 0.00
DAYS_ENTRY_PAYMENT 전체 평균: -1051.11, 극단치 제외 평균: -1051.11, 차이: 0.00
AMT_INSTALMENT 전체 평균: 17050.91, 극단치 제외 평균: 9824.22, 차이: 7226.68
AMT_PAYMENT 전체 평균: 17238.22, 극단치 제외 평균: 9211.85, 차이: 8026.37

=== 상위 1% 제거 방식 ===
NUM_INSTALMENT_NUMBER 전체 평균: 18.87, 상위 1% 제외 평균: 17.62, 차이: 1.25
DAYS_INSTALMENT 전체 평균: -1042.27, 상위 1% 제외 평균: -1052.17, 차이: 9.90
DAYS_ENTRY_PAYMENT 전체 평균: -1051.11, 상위 1% 제외 평균: -1061.03, 차이: 9.91
AMT_INSTALMENT 전체 평균: 17050.91, 상위 1% 제외 평균: 13207.94, 차이: 3842.97
AMT_PAYMENT 전체 평균: 17238.22, 상위 1% 제외 평균: 12938.28, 차이: 4299.94


In [7]:
cols_to_remove = ['AMT_INSTALMENT', 'AMT_PAYMENT']
outlier_dict = {}

for col in cols_to_remove:
    upper = df_install[col].quantile(0.99)
    ids = df_install.loc[df_install[col] > upper, 'SK_ID_CURR'].unique()
    outlier_dict[col] = ids

# 데이터프레임으로 변환 (컬럼별로 아이디를 리스트로 저장)
outlier_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in outlier_dict.items()]))

# csv로 저장
outlier_df.to_csv("install_outlier_ids_by_column.csv", index=False)

In [8]:
cols_to_remove = ['AMT_INSTALMENT', 'AMT_PAYMENT']
outlier_ids = set()

for col in cols_to_remove:
    upper = df_install[col].quantile(0.99)
    ids = df_install.loc[df_install[col] > upper, 'SK_ID_CURR'].unique()
    outlier_ids.update(ids)

print(f"중복 없는 SK_ID_CURR 개수: {len(outlier_ids)}")
print(list(outlier_ids)[:10])  # 일부만 출력

중복 없는 SK_ID_CURR 개수: 79851
[262145, 262147, 262150, 262157, 262160, 262162, 262169, 262175, 262176, 262179]


In [14]:
df_install.shape

(13605401, 8)

In [15]:
df_install_top1_clean.to_csv("installments_payments_fix.csv", index=False)
df_credit_top1_clean.to_csv("credit_card_balance_fix.csv", index=False)

In [28]:
import pandas as pd
import numpy as np
from functools import reduce

# Step 0: 복사본 만들고 파생 컬럼 생성
ccb = df_credit_top1_clean.copy()
ccb = ccb[(ccb['MONTHS_BALANCE'] >= -35) & (ccb['MONTHS_BALANCE'] <= 0)]  # 최근 12개월

# 파생 비율 계산
ccb['PAYMENT_OVER_MIN'] = ccb['AMT_PAYMENT_CURRENT'] / ccb['AMT_INST_MIN_REGULARITY'].replace(0, np.nan)
ccb['REPAY_OVER_BALANCE'] = ccb['AMT_PAYMENT_CURRENT'] / ccb['AMT_BALANCE'].replace(0, np.nan)
ccb['UTILIZATION_RATIO'] = ccb['AMT_BALANCE'] / ccb['AMT_CREDIT_LIMIT_ACTUAL'].replace(0, np.nan)

ccb['ATM_RATIO'] = ccb['AMT_DRAWINGS_ATM_CURRENT'] / ccb['AMT_DRAWINGS_CURRENT'].replace(0, np.nan)
ccb['POS_RATIO'] = ccb['AMT_DRAWINGS_POS_CURRENT'] / ccb['AMT_DRAWINGS_CURRENT'].replace(0, np.nan)
ccb['OTHER_RATIO'] = ccb['AMT_DRAWINGS_OTHER_CURRENT'] / ccb['AMT_DRAWINGS_CURRENT'].replace(0, np.nan)

# Step 1. KPI 요약
df_kpi = ccb.groupby('SK_ID_CURR').agg(
    AVG_SK_DPD=('SK_DPD', 'mean'),
    MAX_SK_DPD=('SK_DPD', 'max'),
    CNT_LATE=('SK_DPD', lambda x: (x > 0).sum()),
    CNT_SERIOUS_LATE=('SK_DPD', lambda x: (x >= 30).sum()),
    AVG_SK_DPD_DEF=('SK_DPD_DEF', 'mean'),

    AVG_MIN_PAY_RATIO=('PAYMENT_OVER_MIN', 'mean'),
    AVG_REPAY_RATIO=('REPAY_OVER_BALANCE', 'mean'),
    AVG_UTIL_RATIO=('UTILIZATION_RATIO', 'mean'),

    ATM_RATIO=('ATM_RATIO', 'mean'),
    POS_RATIO=('POS_RATIO', 'mean'),
    OTHER_RATIO=('OTHER_RATIO', 'mean'),

    CNT_DRAWINGS_SUM=('CNT_DRAWINGS_CURRENT', 'sum'),
    CNT_DRAWINGS_AVG=('CNT_DRAWINGS_CURRENT', 'mean')
)

# Step 2. 수치형 컬럼 전체 요약
numeric_cols = [
    'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_TOTAL_RECEIVABLE',
    'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT',
    'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT',
]

df_numeric = ccb.groupby('SK_ID_CURR')[numeric_cols].agg(['mean', 'std', 'max', 'min'])
df_numeric.columns = ['_'.join(col) for col in df_numeric.columns]
df_numeric.reset_index(inplace=True)

# Step 3. 계약 상태 요약
contract_status = ccb.groupby(['SK_ID_CURR', 'NAME_CONTRACT_STATUS']).size().unstack(fill_value=0)
contract_status.columns = [f'STATUS_{col}' for col in contract_status.columns]

# Step 4. 활동 중인 카드 수
active_cards = df_credit_top1_clean[
    df_credit_top1_clean['NAME_CONTRACT_STATUS'] == 'Active'
].groupby('SK_ID_CURR')['SK_ID_PREV'].nunique().rename('CNT_ACTIVE_CARDS')

# Step 5. 통합
df_summary = reduce(lambda left, right: left.join(right, how='outer'),
    [df_kpi, df_numeric.set_index('SK_ID_CURR'), contract_status, active_cards])

df_summary.reset_index(inplace=True)


In [ ]:
df_summary

In [29]:
df_summary.columns

Index(['SK_ID_CURR', 'AVG_SK_DPD', 'MAX_SK_DPD', 'CNT_LATE',
       'CNT_SERIOUS_LATE', 'AVG_SK_DPD_DEF', 'AVG_MIN_PAY_RATIO',
       'AVG_REPAY_RATIO', 'AVG_UTIL_RATIO', 'ATM_RATIO', 'POS_RATIO',
       'OTHER_RATIO', 'CNT_DRAWINGS_SUM', 'CNT_DRAWINGS_AVG',
       'AMT_BALANCE_mean', 'AMT_BALANCE_std', 'AMT_BALANCE_max',
       'AMT_BALANCE_min', 'AMT_CREDIT_LIMIT_ACTUAL_mean',
       'AMT_CREDIT_LIMIT_ACTUAL_std', 'AMT_CREDIT_LIMIT_ACTUAL_max',
       'AMT_CREDIT_LIMIT_ACTUAL_min', 'AMT_TOTAL_RECEIVABLE_mean',
       'AMT_TOTAL_RECEIVABLE_std', 'AMT_TOTAL_RECEIVABLE_max',
       'AMT_TOTAL_RECEIVABLE_min', 'AMT_DRAWINGS_CURRENT_mean',
       'AMT_DRAWINGS_CURRENT_std', 'AMT_DRAWINGS_CURRENT_max',
       'AMT_DRAWINGS_CURRENT_min', 'AMT_DRAWINGS_ATM_CURRENT_mean',
       'AMT_DRAWINGS_ATM_CURRENT_std', 'AMT_DRAWINGS_ATM_CURRENT_max',
       'AMT_DRAWINGS_ATM_CURRENT_min', 'AMT_INST_MIN_REGULARITY_mean',
       'AMT_INST_MIN_REGULARITY_std', 'AMT_INST_MIN_REGULARITY_max',
       'AMT_I

In [30]:
df_summary.shape

(66012, 47)

In [19]:
inst = df_install_top1_clean.copy()

# 연체일수 계산
inst['PAYMENT_DELAY'] = inst['DAYS_ENTRY_PAYMENT'] - inst['DAYS_INSTALMENT']  # 양수면 연체

# 완납 여부: 실제 납부 ≥ 예정 납부
inst['IS_FULL_PAYMENT'] = (inst['AMT_PAYMENT'] >= inst['AMT_INSTALMENT']).astype(int)

# 지연 여부: 연체일수 > 0
inst['IS_LATE'] = (inst['PAYMENT_DELAY'] > 0).astype(int)

# 조기 납부 여부
inst['IS_EARLY'] = (inst['PAYMENT_DELAY'] < 0).astype(int)

# 부족 납부 여부: 실제 납부 < 예정 납부
inst['IS_UNDERPAID'] = (inst['AMT_PAYMENT'] < inst['AMT_INSTALMENT']).astype(int)

# 이중 리스크: 늦게 내고 덜 낸 경우
inst['IS_LATE_AND_UNDER'] = ((inst['IS_LATE'] == 1) & (inst['IS_UNDERPAID'] == 1)).astype(int)


In [20]:
df_installment_kpi = inst.groupby('SK_ID_CURR').agg(
    avg_payment_delay=('PAYMENT_DELAY', lambda x: x[x > 0].mean()),  # 연체일수 평균 (양수만)
    pct_late_payments=('IS_LATE', 'mean'),       # 연체 비율
    pct_full_payments=('IS_FULL_PAYMENT', 'mean'),  # 완납 비율
    pct_early_payments=('IS_EARLY', 'mean'),     # 조기납부 비율
    pct_underpaid=('IS_UNDERPAID', 'mean'),      # 부족납부 비율
    pct_late_and_under=('IS_LATE_AND_UNDER', 'mean')  # 이중 리스크 비율
)

df_installment_kpi.reset_index(inplace=True)


In [21]:
df_installment_kpi.shape

(339069, 7)

In [22]:
df_install_top1_clean.shape

(13331828, 8)

In [27]:
df_installment_kpi.columns

Index(['SK_ID_CURR', 'avg_payment_delay', 'pct_late_payments',
       'pct_full_payments', 'pct_early_payments', 'pct_underpaid',
       'pct_late_and_under'],
      dtype='object')

In [14]:
df_trian = pd.read_csv(r"C:\Users\rtc32\OneDrive\바탕 화면\데이톤\Datasets\application_train.csv")

In [15]:
df_train = df_trian[['SK_ID_CURR', 'TARGET']]

In [16]:
df_credit_kpi = df_summary.merge(df_train, on='SK_ID_CURR', how='left')

In [17]:
df_installment_kpi = df_installment_kpi.merge(df_train, on='SK_ID_CURR', how='left')

In [35]:
df_credit_kpi.columns

Index(['SK_ID_CURR', 'AVG_SK_DPD', 'MAX_SK_DPD', 'CNT_LATE',
       'CNT_SERIOUS_LATE', 'AVG_SK_DPD_DEF', 'AVG_MIN_PAY_RATIO',
       'AVG_REPAY_RATIO', 'AVG_UTIL_RATIO', 'ATM_RATIO', 'POS_RATIO',
       'OTHER_RATIO', 'CNT_DRAWINGS_SUM', 'CNT_DRAWINGS_AVG',
       'AMT_BALANCE_mean', 'AMT_BALANCE_std', 'AMT_BALANCE_max',
       'AMT_BALANCE_min', 'AMT_CREDIT_LIMIT_ACTUAL_mean',
       'AMT_CREDIT_LIMIT_ACTUAL_std', 'AMT_CREDIT_LIMIT_ACTUAL_max',
       'AMT_CREDIT_LIMIT_ACTUAL_min', 'AMT_TOTAL_RECEIVABLE_mean',
       'AMT_TOTAL_RECEIVABLE_std', 'AMT_TOTAL_RECEIVABLE_max',
       'AMT_TOTAL_RECEIVABLE_min', 'AMT_DRAWINGS_CURRENT_mean',
       'AMT_DRAWINGS_CURRENT_std', 'AMT_DRAWINGS_CURRENT_max',
       'AMT_DRAWINGS_CURRENT_min', 'AMT_DRAWINGS_ATM_CURRENT_mean',
       'AMT_DRAWINGS_ATM_CURRENT_std', 'AMT_DRAWINGS_ATM_CURRENT_max',
       'AMT_DRAWINGS_ATM_CURRENT_min', 'AMT_INST_MIN_REGULARITY_mean',
       'AMT_INST_MIN_REGULARITY_std', 'AMT_INST_MIN_REGULARITY_max',
       'AMT_I

In [36]:
df_installment_kpi.columns

Index(['SK_ID_CURR', 'avg_payment_delay', 'pct_late_payments',
       'pct_full_payments', 'pct_early_payments', 'pct_underpaid',
       'pct_late_and_under', 'TARGET'],
      dtype='object')

In [37]:
df_credit_kpi.to_csv("credit_kpi.csv", index=False)
df_installment_kpi.to_csv("installment_kpi.csv", index=False)

In [10]:
import pandas as pd
import numpy as np
from functools import reduce

# Step 0: 복사본 만들고 파생 컬럼 생성
ccb = df_credit.copy()
ccb = ccb[(ccb['MONTHS_BALANCE'] >= -35) & (ccb['MONTHS_BALANCE'] <= 0)]  # 최근 12개월

# 파생 비율 계산
ccb['PAYMENT_OVER_MIN'] = ccb['AMT_PAYMENT_CURRENT'] / ccb['AMT_INST_MIN_REGULARITY'].replace(0, np.nan)
ccb['REPAY_OVER_BALANCE'] = ccb['AMT_PAYMENT_CURRENT'] / ccb['AMT_BALANCE'].replace(0, np.nan)
ccb['UTILIZATION_RATIO'] = ccb['AMT_BALANCE'] / ccb['AMT_CREDIT_LIMIT_ACTUAL'].replace(0, np.nan)

ccb['ATM_RATIO'] = ccb['AMT_DRAWINGS_ATM_CURRENT'] / ccb['AMT_DRAWINGS_CURRENT'].replace(0, np.nan)
ccb['POS_RATIO'] = ccb['AMT_DRAWINGS_POS_CURRENT'] / ccb['AMT_DRAWINGS_CURRENT'].replace(0, np.nan)
ccb['OTHER_RATIO'] = ccb['AMT_DRAWINGS_OTHER_CURRENT'] / ccb['AMT_DRAWINGS_CURRENT'].replace(0, np.nan)

# Step 1. KPI 요약
df_kpi = ccb.groupby('SK_ID_CURR').agg(
    AVG_SK_DPD=('SK_DPD', 'mean'),
    MAX_SK_DPD=('SK_DPD', 'max'),
    CNT_LATE=('SK_DPD', lambda x: (x > 0).sum()),
    CNT_SERIOUS_LATE=('SK_DPD', lambda x: (x >= 30).sum()),
    AVG_SK_DPD_DEF=('SK_DPD_DEF', 'mean'),

    AVG_MIN_PAY_RATIO=('PAYMENT_OVER_MIN', 'mean'),
    AVG_REPAY_RATIO=('REPAY_OVER_BALANCE', 'mean'),
    AVG_UTIL_RATIO=('UTILIZATION_RATIO', 'mean'),

    ATM_RATIO=('ATM_RATIO', 'mean'),
    POS_RATIO=('POS_RATIO', 'mean'),
    OTHER_RATIO=('OTHER_RATIO', 'mean'),

    CNT_DRAWINGS_SUM=('CNT_DRAWINGS_CURRENT', 'sum'),
    CNT_DRAWINGS_AVG=('CNT_DRAWINGS_CURRENT', 'mean')
)

# Step 2. 수치형 컬럼 전체 요약
numeric_cols = [
    'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_TOTAL_RECEIVABLE',
    'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT',
    'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT',
]

df_numeric = ccb.groupby('SK_ID_CURR')[numeric_cols].agg(['mean', 'std', 'max', 'min'])
df_numeric.columns = ['_'.join(col) for col in df_numeric.columns]
df_numeric.reset_index(inplace=True)

# Step 3. 계약 상태 요약
contract_status = ccb.groupby(['SK_ID_CURR', 'NAME_CONTRACT_STATUS']).size().unstack(fill_value=0)
contract_status.columns = [f'STATUS_{col}' for col in contract_status.columns]

# Step 4. 활동 중인 카드 수
active_cards = df_credit[
    df_credit['NAME_CONTRACT_STATUS'] == 'Active'
].groupby('SK_ID_CURR')['SK_ID_PREV'].nunique().rename('CNT_ACTIVE_CARDS')

# Step 5. 통합
df_summary = reduce(lambda left, right: left.join(right, how='outer'),
    [df_kpi, df_numeric.set_index('SK_ID_CURR'), contract_status, active_cards])

df_summary.reset_index(inplace=True)


In [11]:
inst = df_install.copy()

# 연체일수 계산
inst['PAYMENT_DELAY'] = inst['DAYS_ENTRY_PAYMENT'] - inst['DAYS_INSTALMENT']  # 양수면 연체

# 완납 여부: 실제 납부 ≥ 예정 납부
inst['IS_FULL_PAYMENT'] = (inst['AMT_PAYMENT'] >= inst['AMT_INSTALMENT']).astype(int)

# 지연 여부: 연체일수 > 0
inst['IS_LATE'] = (inst['PAYMENT_DELAY'] > 0).astype(int)

# 조기 납부 여부
inst['IS_EARLY'] = (inst['PAYMENT_DELAY'] < 0).astype(int)

# 부족 납부 여부: 실제 납부 < 예정 납부
inst['IS_UNDERPAID'] = (inst['AMT_PAYMENT'] < inst['AMT_INSTALMENT']).astype(int)

# 이중 리스크: 늦게 내고 덜 낸 경우
inst['IS_LATE_AND_UNDER'] = ((inst['IS_LATE'] == 1) & (inst['IS_UNDERPAID'] == 1)).astype(int)

In [12]:
df_installment_kpi = inst.groupby('SK_ID_CURR').agg(
    avg_payment_delay=('PAYMENT_DELAY', lambda x: x[x > 0].mean()),  # 연체일수 평균 (양수만)
    pct_late_payments=('IS_LATE', 'mean'),       # 연체 비율
    pct_full_payments=('IS_FULL_PAYMENT', 'mean'),  # 완납 비율
    pct_early_payments=('IS_EARLY', 'mean'),     # 조기납부 비율
    pct_underpaid=('IS_UNDERPAID', 'mean'),      # 부족납부 비율
    pct_late_and_under=('IS_LATE_AND_UNDER', 'mean')  # 이중 리스크 비율
)

df_installment_kpi.reset_index(inplace=True)


In [18]:
df_credit_kpi.to_csv("credit_kpi_all.csv", index=False)
df_installment_kpi.to_csv("installment_kpi_all.csv", index=False)

In [4]:
df_credit_kpi = pd.read_csv("credit_kpi.csv")

In [3]:
avg_card_months = df_credit.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['MONTHS_BALANCE'].count().reset_index()
avg_by_customer = avg_card_months.groupby('SK_ID_CURR')['MONTHS_BALANCE'].mean().reset_index()
avg_by_customer.columns = ['SK_ID_CURR', 'AVG_CARD_USAGE_MONTHS']

In [5]:
df_credit_kpi = df_credit_kpi.merge(avg_by_customer, on='SK_ID_CURR', how='left')

In [ ]:
df_credit_kpi.to_csv("credit_kpi2.csv", index=False)

: 